In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import numexpr as ne
import time
import copy
import sys

from tqdm import tqdm
from sklearn.metrics import accuracy_score, log_loss

import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../pytorch_utils/')
from utils import get_data, top5_accuracy, per_class_accuracy

In [ ]:
import torch.cuda
torch.cuda.is_available()

In [ ]:
torch.backends.cudnn.benchmark = True

# Load data

In [ ]:
%%time
X_train, X_val, Y_train, Y_val = get_data()
train_size = len(X_train)
val_size = len(X_val)
print(train_size, val_size)

# Create iterators

In [ ]:
val_data = TensorDataset(
    torch.FloatTensor(X_val), 
    torch.LongTensor(Y_val)
)

val_iterator = DataLoader(
    val_data, batch_size=64, shuffle=False, pin_memory=True
)

In [ ]:
train_data = TensorDataset(
    torch.FloatTensor(X_train), 
    torch.LongTensor(Y_train)
)

train_iterator = DataLoader(
    train_data, batch_size=64, shuffle=False, pin_memory=True
)

# Get models

In [ ]:
sys.path.append('../densenet/')
sys.path.append('../resnet/')
sys.path.append('../inception_v3/')

from model_densenet import make_model as make_densenet
from model_resnet import make_model as make_resnet
from model_inception import make_model as make_inception

In [ ]:
densenet, _, _ = make_densenet()
densenet = densenet.cpu()

resnet, _, _ = make_resnet()
resnet = resnet.cpu()

inception, _, _ = make_inception()
inception = inception.cpu()

In [ ]:
all_models = [
    (densenet, '../densenet/model_state0.pytorch'),
    (resnet, '../resnet/model_state0.pytorch'),
    (inception, '../inception_v3/model_state0.pytorch')
]

# Try ensemble

In [ ]:
ensemble_predictions = {}

In [ ]:
%%time
for i, (model, state) in enumerate(all_models):
    
    model.load_state_dict(torch.load(state))
    model = model.cuda()
    ensemble_predictions[i] = []
    model.eval()
    
    for x_batch, _ in tqdm(val_iterator):

        x_batch = Variable(x_batch.cuda(), volatile=True)
        logits = model(x_batch)

        # compute probabilities
        probs = F.softmax(logits) 
        ensemble_predictions[i] += [probs.cpu().data.numpy()]
    
    model = model.cpu()

In [ ]:
ensemble_predictions = {
    p: np.concatenate(ensemble_predictions[p], axis=0) 
    for p in ensemble_predictions
}

In [ ]:
predictions = np.stack(ensemble_predictions.values()).mean(0)

In [ ]:
accuracy_score(Y_val, predictions.argmax(1))

In [ ]:
log_loss(Y_val, predictions)

In [ ]:
top5_accuracy(Y_val, predictions)

In [ ]:
per_class_acc = per_class_accuracy(Y_val, predictions)
per_class_acc

# Error analysis

In [ ]:
erroneous = Y_val != predictions.argmax(1)
n_errors = len(Y_val[erroneous])
n_errors

In [ ]:
to_show = np.random.choice(np.arange(0, n_errors), size=30, replace=False)

In [ ]:
pictures = X_val[erroneous][to_show].copy()
pictures_predictions = predictions.argmax(1)[erroneous][to_show]
pictures_probs = predictions.max(1)[erroneous][to_show]
pictures_true = Y_val[erroneous][to_show]

In [ ]:
mean = np.array([0.485, 0.456, 0.406], dtype='float32')
std = np.array([0.229, 0.224, 0.225], dtype='float32')
decode = np.load('../utils/decode.npy')[()]

In [ ]:
pictures = np.transpose(pictures, axes=(0, 2, 3, 1))
ne.evaluate('pictures*std', out=pictures);
ne.evaluate('pictures + mean', out=pictures);

In [ ]:
# show pictures, predicted classes and probabilities
_, axes = plt.subplots(nrows=6, ncols=5, figsize=(14, 16))
axes = axes.flatten()
for i, pic in enumerate(pictures):
    axes[i].set_axis_off();
    axes[i].imshow(pic);
    title = decode[pictures_predictions[i] + 1] + ' ' +\
        str(pictures_probs[i]) + '\ntrue: ' + decode[pictures_true[i] + 1]
    axes[i].set_title(title);
plt.tight_layout()

# Get logits

In [ ]:
train_logits = {}

In [ ]:
%%time
for i, (model, state) in enumerate(all_models):
    
    model.load_state_dict(torch.load(state))
    model = model.cuda()
    train_logits[i] = []
    model.eval()
    
    for x_batch, _ in tqdm(train_iterator):

        x_batch = Variable(x_batch.cuda(), volatile=True)
        logits = model(x_batch)
        train_logits[i] += [logits.cpu().data.numpy()]
        
    model = model.cpu()

In [ ]:
train_logits = {
    p: np.concatenate(train_logits[p], axis=0) 
    for p in train_logits
}

In [ ]:
np.save('/home/ubuntu/data/train_logits.npy', train_logits)

In [ ]:
train_logits[0].shape